In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Masters_Project/Datasets/OASIS3/ganomaly_pytorch

/content/drive/MyDrive/Masters_Project/Datasets/OASIS3/ganomaly_pytorch


# TODO
Fix issue causing multiclass labels to be sent to evaluation class

In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 102kB 6.5MB/s 
     |████████████████████████████████| 163kB 13.2MB/s 
     |████████████████████████████████| 440kB 21.4MB/s 
     |████████████████████████████████| 2.3MB 38.2MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
     |████████████████████████████████| 286kB 43.6MB/s 
     |████████████████████████████████| 92kB 12.2MB/s 
     |████████████████████████████████| 13.1MB 25.7MB/s 
     |████████████████████████████████| 17.3MB 302kB/s 
     |████████████████████████████████| 112kB 55.7MB/s 
     |████████████████████████████████| 163kB 49.5MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
     |████████████████████████████████| 235kB 57.5MB/s 
     |████████████████████████████████| 512kB 45.3MB/s 
     |████████████████████████████████| 1.1MB 51.2MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
     |████████████████████████████████| 6.7MB 41.1MB/s 


In [4]:
!pip install mkl-fft

     |████████████████████████████████| 245kB 8.1MB/s 
     |████████████████████████████████| 171.6MB 85kB/s 
     |████████████████████████████████| 31.6MB 175kB/s 
     |████████████████████████████████| 169.0MB 102kB/s 
     |████████████████████████████████| 3.9MB 48.7MB/s 


In [ ]:
#!pip install Pillow==6.2.2

  Using cached https://files.pythonhosted.org/packages/c3/3f/03375124676ab49ca6e6917c0f1f663afb8354d5d24e12f4fe4587a39ae2/Pillow-6.2.2-cp37-cp37m-manylinux1_x86_64.whl
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 8.1.0
    Uninstalling Pillow-8.1.0:
      Successfully uninstalled Pillow-8.1.0


In [5]:
import os
import torch
import numpy as np
import torchvision.datasets as datasets
from torchvision.datasets import MNIST
from torchvision.datasets import CIFAR10
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

from PIL import Image

from options import Options
from lib.data import load_data
from lib.model import Ganomaly

In [6]:
# params
OPT_isize = 32
OPT_abnormal_class = "plane"
OPT_manualseed = 55
OPT_batchsize = 64
OPT_workers = 8

splits = ['train', 'test']
drop_last_batch = {'train': True, 'test': False}
shuffle = {'train': True, 'test': False}

In [7]:
transform = transforms.Compose(
    [
        transforms.Resize(OPT_isize),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]
)

classes = {
    'plane': 0, 'car': 1, 'bird': 2, 'cat': 3, 'deer': 4,
    'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9
}

In [8]:
dataset = {}
dataset['train'] = CIFAR10(root='./data', train=True, download=True, transform=transform)
dataset['test'] = CIFAR10(root='./data', train=False, download=True, transform=transform)

0it [00:00, ?it/s]

170500096it [00:03, 43407655.74it/s]                               


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [9]:
type(dataset['train'].targets)

list

In [10]:
##
def get_cifar_anomaly_dataset(trn_img, trn_lbl, tst_img, tst_lbl, abn_cls_idx=0, manualseed=-1):
    """[summary]
    Arguments:
        trn_img {np.array} -- Training images
        trn_lbl {np.array} -- Training labels
        tst_img {np.array} -- Test     images
        tst_lbl {np.array} -- Test     labels
    Keyword Arguments:
        abn_cls_idx {int} -- Anomalous class index (default: {0})
    Returns:
        [np.array] -- New training-test images and labels.
    """
    # Convert train-test labels into numpy array.
    trn_lbl = np.array(trn_lbl)
    tst_lbl = np.array(tst_lbl)
    print(trn_lbl)

    # --
    # Find idx, img, lbl for abnormal and normal on org dataset.
    nrm_trn_idx = np.where(trn_lbl != abn_cls_idx)[0]
    abn_trn_idx = np.where(trn_lbl == abn_cls_idx)[0]
    print(type(nrm_trn_idx))
    print(nrm_trn_idx)
    nrm_trn_img = trn_img[nrm_trn_idx]    # Normal training images
    abn_trn_img = trn_img[abn_trn_idx]    # Abnormal training images
    nrm_trn_lbl = trn_lbl[nrm_trn_idx]    # Normal training labels
    abn_trn_lbl = trn_lbl[abn_trn_idx]    # Abnormal training labels.

    nrm_tst_idx = np.where(tst_lbl != abn_cls_idx)[0]
    abn_tst_idx = np.where(tst_lbl == abn_cls_idx)[0]
    nrm_tst_img = tst_img[nrm_tst_idx]    # Normal training images
    abn_tst_img = tst_img[abn_tst_idx]    # Abnormal training images.
    nrm_tst_lbl = tst_lbl[nrm_tst_idx]    # Normal training labels
    abn_tst_lbl = tst_lbl[abn_tst_idx]    # Abnormal training labels.

    # --
    # Assign labels to normal (0) and abnormals (1)
    nrm_trn_lbl[:] = 0
    nrm_tst_lbl[:] = 0
    abn_trn_lbl[:] = 1
    abn_tst_lbl[:] = 1

    # --
    if manualseed != -1:
        # Random seed.
        # Concatenate the original train and test sets.
        nrm_img = np.concatenate((nrm_trn_img, nrm_tst_img), axis=0)
        nrm_lbl = np.concatenate((nrm_trn_lbl, nrm_tst_lbl), axis=0)
        abn_img = np.concatenate((abn_trn_img, abn_tst_img), axis=0)
        abn_lbl = np.concatenate((abn_trn_lbl, abn_tst_lbl), axis=0)

        # Split the normal data into the new train and tests.
        idx = np.arange(len(nrm_lbl))
        np.random.seed(manualseed)
        np.random.shuffle(idx)

        nrm_trn_len = int(len(idx) * 0.80)
        nrm_trn_idx = idx[:nrm_trn_len]
        nrm_tst_idx = idx[nrm_trn_len:]

        nrm_trn_img = nrm_img[nrm_trn_idx]
        nrm_trn_lbl = nrm_lbl[nrm_trn_idx]
        nrm_tst_img = nrm_img[nrm_tst_idx]
        nrm_tst_lbl = nrm_lbl[nrm_tst_idx]

    # Create new anomaly dataset based on the following data structure:
    # - anomaly dataset
    #   . -> train
    #        . -> normal
    #   . -> test
    #        . -> normal
    #        . -> abnormal
    new_trn_img = np.copy(nrm_trn_img)
    new_trn_lbl = np.copy(nrm_trn_lbl)
    new_tst_img = np.concatenate((nrm_tst_img, abn_trn_img, abn_tst_img), axis=0)
    new_tst_lbl = np.concatenate((nrm_tst_lbl, abn_trn_lbl, abn_tst_lbl), axis=0)

    return new_trn_img, new_trn_lbl, new_tst_img, new_tst_lbl

In [11]:
train_data, train_targets, \
test_data, test_targets = get_cifar_anomaly_dataset(
    trn_img=dataset['train'].data,
    trn_lbl=dataset['train'].targets,
    tst_img=dataset['test'].data,
    tst_lbl=dataset['test'].targets,
    abn_cls_idx=classes[OPT_abnormal_class],
    manualseed=OPT_manualseed
)

[6 9 9 ... 9 1 1]
<class 'numpy.ndarray'>
[    0     1     2 ... 49997 49998 49999]


In [12]:
dataloader = {x: torch.utils.data.DataLoader(dataset=dataset[x],
                                             batch_size=OPT_batchsize,
                                             shuffle=shuffle[x],
                                             num_workers=int(OPT_workers),
                                             drop_last=drop_last_batch[x],
                                             worker_init_fn=(None if OPT_manualseed == -1
                                             else lambda x: np.random.seed(OPT_manualseed)))
              for x in splits}

In [13]:
type(dataloader)

dict

In [14]:
dataloader.keys()

dict_keys(['train', 'test'])

In [15]:
dir(dataloader['train'])

['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_index_sampler',
 'batch_sampler',
 'batch_size',
 'collate_fn',
 'dataset',
 'dataset_kind',
 'drop_last',
 'multiprocessing_context',
 'num_workers',
 'pin_memory',
 'sampler',
 'timeout',
 'worker_init_fn']

In [16]:
print(type(dataloader))
type(dataloader['train'])

<class 'dict'>


torch.utils.data.dataloader.DataLoader

In [17]:
# iterate through and print few of the data types stored within the iterator to examine it
# Create a similar iterator for the OASIS-3 no-ad scans directory

In [18]:
!ls

data	     lib	 options.pyc  README.md
experiments  LICENSE	 output       requirements.txt
__init__.py  options.py  __pycache__  train.py


In [19]:
class Option:
    def __init__(self):
        # Base
        self.dataset = 'cifar10'
        self.dataroot = ''
        self.batchsize = 64
        self.workers = 8
        self.droplast = True
        self.isize = 32
        self.nc = 3
        self.nz = 100
        self.ngf = 64
        self.ndf = 64
        self.extralayers = 0
        self.device = 'gpu'
        self.gpu_ids = '0'
        self.ngpu = 1
        self.name = 'pytorch_ganomaly'
        self.model = 'ganomaly'
        self.display_server = "http://localhost"
        self.display_port = 8097
        self.display_id = 0
        self.display = False
        self.outf = './pyt_ganomaly_output'
        self.manualseed = -1
        self.abnormal_class = 'car'
        self.proportion = 0.1
        self.metric = 'auprc' #roc

        ##
        # Train
        self.print_freq = 100
        self.save_image_freq = 100
        self.save_test_images = True
        self.load_weights = False
        self.resume = ''
        self.phase = 'train'
        self.iter = 0
        self.niter = 15
        self.beta1 = 0.5
        self.lr = 0.0002
        self.w_adv = 1
        self.w_con = 50
        self.w_enc = 1
        self.isTrain = True

opt = Option()

In [20]:
model = Ganomaly(opt, dataloader)
model.train()

  0%|          | 0/781 [00:00<?, ?it/s]

>> Training model Ganomaly.


>> Training model Ganomaly. Epoch 1/15


ValueError: ignored